# ACMA Scrapper
##### ©Haris Hassan

In [16]:
##=============================================================================
## ACMAscrapper V4.0.2
##=============================================================================
##
## Scrapper for Register of Radiocommunications Licences of Australian Communications and Media Authority
# 
#Author Haris Hassan
#Email haris.hassan@radhaz.com.au 
#linkedin https://www.linkedin.com/in/hassanharis/
#
##=============================================================================
# Import libraries
import requests
import string
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)
#Reset index of dataframe after sorting


In [18]:
####Replace Site URL
site_url = 'https://web.acma.gov.au/rrl/site_search.site_lookup?pSITE_ID=36185'

In [19]:
ACMApage = requests.get(site_url)
soup = BeautifulSoup(ACMApage.text, 'lxml')

try:
    SiteDetailsTitle = [td.text for td in soup.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)')]
    SiteDetails = [td.text.strip() for td in soup.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
except Exception as e:
    print (e)
    pass

In [20]:
SiteDetailsDictionary = {SiteDetailsTitle[i]: SiteDetails[i] for i in range(len(SiteDetailsTitle))}
SiteDetailsDictionary['Location'] = ' '.join(SiteDetailsDictionary['Location'].split())

#SiteData = pd.DataFrame([SiteDetailsDictionary])
#display(SiteDetailsDictionary['Location'])

In [21]:
#Check if there's another Page
NextPageLink = ''.join(['https://web.acma.gov.au' + x for x in  [tl.get('href') for tl in soup.findAll('a',{'title':"Next Page"})]])

In [22]:
ACMAtable = soup.find('table',{"class": "tablelist responsive"})
ACMAheaders = [td.text.strip() for td in ACMAtable.select('th')]
ACMAdata = pd.DataFrame(columns = ACMAheaders)

ACMAlinks = []
for j in ACMAtable.find_all('tr')[1:]:
    ACMAValues = [tv.text.strip() for tv in j.find_all('td')]
    ACMAdata.loc[len(ACMAdata)] = ACMAValues
    ACMAlinks.append([tl.get('href') for tl in j.find_all('a')][0])
ACMAdata.insert(1,'links', ACMAlinks )

In [23]:
#Next Page
if NextPageLink and NextPageLink.strip():
    ACMApagep2 = requests.get(NextPageLink)
    soupp2 = BeautifulSoup(ACMApagep2.text, 'lxml')

    ACMAtablep2 = soupp2.find('table',{"class": "tablelist responsive"})
    ACMAheadersp2 = [td.text.strip() for td in ACMAtablep2.select('th')]
    ACMAdatap2 = pd.DataFrame(columns = ACMAheadersp2)

    ACMAlinksp2 = []
    for j in ACMAtablep2.find_all('tr')[1:]:
        ACMAValuesp2 = [tv.text.strip() for tv in j.find_all('td')]
        ACMAdatap2.loc[len(ACMAdatap2)] = ACMAValuesp2
        ACMAlinksp2.append([tl.get('href') for tl in j.find_all('a')][0])
    ACMAdatap2.insert(1,'links', ACMAlinksp2 )

    ACMAdata_final = pd.concat([ACMAdata, ACMAdatap2], axis=0)
else:
    ACMAdata_final = ACMAdata
#Reset index of dataframe after sorting
ACMAdata_final = ACMAdata_final.reset_index(drop=True)

In [24]:
#removing duplicates
ACMAdatalinks = list( dict.fromkeys(ACMAdata_final['links']) )
display(ACMAdata_final)

,ID,links,Frequency,Emission Designator,Azimuth,T/R,Client,BSL/Licence No
0,973276,/rrl/assignment_search.lookup?pEFL_ID=973276,404.24375 MHz,10K1F3E,217.88°,T,Volunteer Marine Rescue NSW (1147602),1972029/1
1,973283,/rrl/assignment_search.lookup?pEFL_ID=973283,413.96875 MHz,10K1F3E,217.88°,R,Volunteer Marine Rescue NSW (1147602),1972030/1
2,896286,/rrl/assignment_search.lookup?pEFL_ID=896286,148.5875 MHz,16K0F2D,,T,NSW Rural Fire Service (5832),1415765/1
3,973284,/rrl/assignment_search.lookup?pEFL_ID=973284,404.51875 MHz,10K1F3E,217.88°,T,Volunteer Marine Rescue NSW (1147602),1972030/1
4,973275,/rrl/assignment_search.lookup?pEFL_ID=973275,413.69375 MHz,10K1F3E,217.88°,R,Volunteer Marine Rescue NSW (1147602),1972029/1


In [25]:
AntennaTableHeader = ['Licence Number', 'Antenna', 'Client','Device Type','Emission Center Frequency','Transmitter Power',
                      'Antenna Height (AGL)', 
                      'Antenna Polarisation','Antenna Azimuth', 'Antenna Tilt','Device Registration ID','Date Authorised',
                      'Destination Link']
AntennaTable = pd.DataFrame(columns = AntennaTableHeader)


for acmalink in ACMAdatalinks:
    url = 'https://web.acma.gov.au'+ acmalink
    page = requests.get(url)
    soup2 = BeautifulSoup(page.text, 'lxml')
    
    try:
        Antennaheaders = [td.text for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
        AntennaValues = [td.text.strip() for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    except Exception as e:
        print (e)
        continue
    
    LinkToList = []
    LinkToTable = soup2.find("table", {"class": "tablelist linked-responsive"})
    if LinkToTable:
        for j in LinkToTable.find_all('tr')[1:]:
            LinkToList.append([tv.text.strip() for tv in j.select('td:nth-of-type(5)')] )
        
    if url[-2:]=='/1':
        try:
            Antennaheaders2 = [td.text for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
            AntennaValues2 = [td.text.strip() for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
        except Exception as e:
            print(e)
            print(acmalink)
            continue
            
    if not LinkToList:
        AntennaValues.extend(['n/a'])
        AntennaValues2.extend(['n/a'])
 
    if LinkToList:
        LinkToListTemp = []
        for x in LinkToList:
            for j in x:
                LinkToListTemp.append([' '.join(p) for p in [j.split()[-5:]]])
        LinkToListTemp=list(dict.fromkeys([''.join(p) for p in LinkToListTemp]))
        for todel in LinkToListTemp:
            if SiteDetailsDictionary['Location'] in todel:
                LinkToListTemp.remove(todel)
        AntennaValues.extend([str(LinkToListTemp)])
        if url[-2:]=='/1':
          AntennaValues2.extend([str(LinkToListTemp)])
    Antennaheaders.extend(['Destination Link'])
    if url[-2:]=='/1':
      Antennaheaders2.extend(['Destination Link'])
    
    
    AntennaDictionary = {Antennaheaders[i]: AntennaValues[i] for i in range(len(Antennaheaders))}
    AntennaTable_toAdd = pd.DataFrame([AntennaDictionary])
    AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd], ignore_index=True)
    
    if url[-2:]=='/1':
        AntennaTable_toAdd2 = pd.DataFrame([{Antennaheaders2[i]: AntennaValues2[i] for i in range(len(Antennaheaders2))}])
        AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd2], ignore_index=True)

In [26]:
#Formatting Data
AntennaTable = AntennaTable.fillna('')
AntennaTable['Antenna']=([''.join(x[2].title() +' '+ x[1] + ' '+ x[0].title()) for x in AntennaTable['Antenna'].str.split(',', 2)])

if 'EFL ID' in AntennaTable:
    AntennaTable['Device Registration ID'] = AntennaTable['Device Registration ID'].astype(str) + AntennaTable['EFL ID']


In [27]:
  
AntennaTable['Antenna']= AntennaTable['Antenna'].str.strip()
AntennaTable['Antenna']= AntennaTable['Antenna'].str.replace('Rf Industries', 'RFI')
AntennaTable['Antenna']= AntennaTable['Antenna'].str.replace('Parallel Array Of Vertical Dipoles', 'Vertical Dipole Array')

AntennaTable['Client']=AntennaTable['Client'].apply(lambda x: x.title())
AntennaTable['Client']=AntennaTable['Client'].str.replace('Limited', 'Ltd')
AntennaTable['Client']=AntennaTable['Client'].str.replace('New South Wales Government Telecommunications Authority', 'NSWTA')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('/', '-')



In [28]:
#Frequency conversion to MHZ
#freq_list = [str(float(x[0])*1000) + ' MHz' if x[1] == 'GHz' else x[0] + ' MHz' for x in AntennaTable['Emission Center Frequency'].str.split(' ')]
#freq_df = pd.DataFrame({'Freq (MHz)': freq_list})
#AntennaTable = pd.concat([AntennaTable, freq_df], axis=1)
#display(AntennaTable['Freq (MHz)'])

In [29]:


Renamed_headers = ['Licence', 'Antenna', 'Client','Type','Frequency','Power','Height', 
                      'Polarisation','Azimuth', 'Tilt','Device ID','Date Authorised','Destination Link']

AntennaTable_export = AntennaTable
AntennaTable_export = AntennaTable_export.rename(columns={'Licence Number': 'Licence', 'Device Type': 'Type', 
                                                          'Emission Center Frequency': 'Frequency'
                                                          ,'Transmitter Power': 'Power'
                                                          , 'Antenna Height (AGL)': 'Height', 
                                                          'Antenna Polarisation': 'Polarisation', 
                                                          'Antenna Azimuth': 'Azimuth', 
                   'Device Registration ID': 'Device ID','Antenna Tilt': 'Tilt'})

#AntennaTable_export = AntennaTable_export.rename(columns= dict(zip(Antennaheaders, Renamed_headers)))


AntennaTable_export = AntennaTable_export.reset_index(drop=True)
AntennaTable_export.index += 1
display(AntennaTable_export[Renamed_headers])


,Licence,Antenna,Client,Type,Frequency,Power,Height,Polarisation,Azimuth,Tilt,Device ID,Date Authorised,Destination Link
1,1972029/1,Polar 326(H) Yagi (Horizontal Polarisation)-Y,Volunteer Marine Rescue Nsw,Transmitter,404.24375 MHz,1 W pY,10 m,Linear Horizontal,217.88°,.06°,973276,29-Jan-2014,['TERREY HILLS NSW 2084 (4247)']
2,1972030/1,Polar 326(H) Yagi (Horizontal Polarisation)-Y,Volunteer Marine Rescue Nsw,Receiver,413.96875 MHz,,10 m,Linear Horizontal,217.88°,.05°,973283,29-Jan-2014,['TERREY HILLS NSW 2084 (4247)']
3,1415765/1,Aea 529 Dipole-D,Nsw Rural Fire Service,Transmitter,148.5875 MHz,100 W pY,10 m,Linear Vertical,,,896286,20-Apr-2000,n/a
4,1972030/1,Polar 326(H) Yagi (Horizontal Polarisation)-Y,Volunteer Marine Rescue Nsw,Transmitter,404.51875 MHz,1 W pY,10 m,Linear Horizontal,217.88°,.05°,973284,29-Jan-2014,['TERREY HILLS NSW 2084 (4247)']
5,1972029/1,Polar 326(H) Yagi (Horizontal Polarisation)-Y,Volunteer Marine Rescue Nsw,Receiver,413.69375 MHz,,10 m,Linear Horizontal,217.88°,.06°,973275,29-Jan-2014,['TERREY HILLS NSW 2084 (4247)']


In [30]:
AntennaTable_export.to_excel(r'C:\Users\Mewtwo\Desktop'+'\\ACMA '+  AntennaDictionary['Site'] + '.xlsx', columns = Renamed_headers, index=True)

In [15]:
AntennaTable_export.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata.html', columns = Renamed_headers, index=True)
import subprocess
subprocess.call('wkhtmltoimage -f png --width 0 Antennadata.html Antennadata.png', shell=True)



1